In [1]:
# 정식일 이후 16주 동안 단위면적당 최소 에너지 비용(누적 난방용 도시가스 사용량)으로
# 최대 수확량을 예측 할 수 있는 환경/생육 모델을 구하세요

# 모델의 성능과 정확도 평가는 학습 모델의 RMSE와 R2 score로 평가

# 1. 코드 제출은 ipynb 파일로만 합니다.
# 시작 후 매 6시간 마다 제출 버튼으로 제출

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from math import sqrt

In [4]:
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')
input_data
# frmDist, date, inTp, inHd, otmsuplyqy, acSlrdQy, cunt, ph, outTp, outWs, daysuplyqy, inCo2, ec, frmYear, frmWeek,
# frtstGrupp, lefstalklt, frtstSetCo, pllnLt, flanGrupp, frtstCo, flanJnt, tcdmt, frmhsFclu, hvstGrupp, hvstJnt, grwtLt,
# fcluHg, lefLt, flwrCo, hvstCo, lefCunt, frtstJnt, lefBt, stemThck, frmAr, frmDov, outtrn_cumsum, WaterUsage, WaterCost,
# FertillizerUsage, FertillizerCost, CO2Usage, CO2Cost, MistUsageTime, Mist Cost, HeatingEnergyUsage_cumsum


# 농가구역, 날짜, 내부온도, 외부온도, 내부습도, 내부Co2, 풍속, 누적 일사량, 급액 EC, 급액 PH, 일 급액횟수, 일 급액량(CC/1주수), 1회 급액량(CC/1주수), 연도, 주차,
# 월, 생장길이, 초장, 옆수, 옆장, 옆폭, 엽병장, 관부직경, 화방높이, 줄기 굵기, 개화화방, 개화군,
# 착과군, 수확군, 개화마디, 착과마디, 수확마디, 꽃개수, 착과수, 수확수, 열매수, 면적, 재식밀도, 누적 생산량, 물 사용량, 물 사용 비용,
# 비료 사용량, 비료 사용 비용, 누적 난방 에너지 사용량, CO2사용량, CO2사용비용, 미스트 사용시간, 미스트 사용비용

# outtrn_cumsum /  HeatingEnergyUsage_cumsum



,frmDist,date,inTp,inHd,otmsuplyqy,acSlrdQy,cunt,ph,outTp,outWs,...,outtrn_cumsum,WaterUsage,WaterCost,FertilizerUsage,FertilizerCost,CO2Usage,CO2Cost,MistUsageTime,Mist Cost,HeatingEnergyUsage_cumsum
0,DBSF1059,20220926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0
1,DBSF1059,20221001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1347.554178,202.133127,4711.308821,5936.249114,0.000000,0.0000,190.168990,2221.173803,0.0
2,DBSF1059,20221002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,69.899353,10.484903,271.379890,341.938662,0.000000,0.0000,0.000000,0.000000,0.0
3,DBSF1059,20221003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,412.225065,61.833760,1539.505534,1939.776973,253.744916,149.7095,0.000000,0.000000,0.0
4,DBSF1059,20221004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,590.608634,88.591295,2614.973165,3294.866187,3032.885084,1789.4022,1.149739,13.428955,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84835,DBSF9969,20180710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,235272.042536,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0
84836,DBSF9969,20180711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,227947.124946,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0
84837,DBSF9969,20180712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,231527.426327,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0
84838,DBSF9969,20180713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,251254.321635,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0


In [6]:
# 데이터 불러오기
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

# 농가구역 One-hot encoding
input_data = pd.get_dummies(input_data, columns=['frmDist'], drop_first=False)

# na값 drop
input_data = input_data.dropna()

# 특성과 타겟 변수 설정
X = input_data.drop(columns=['outtrn_cumsum', 'HeatingEnergyUsage_cumsum'])
y = input_data[['outtrn_cumsum', 'HeatingEnergyUsage_cumsum']]

# 트레이닝, 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Extra Trees Regressor 모델
model = ExtraTreesRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

# 평가
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R-squared (R2):", r2)

RMSE: 29883.836819457
R-squared (R2): 0.9888673562575706
